# Collect all Commit Changes and match the Functions

## remove comments and empty lines

In [34]:
import re
def stripComments(code):
    code = str(code)
    code = re.sub(r'^$\n', '', code, flags=re.MULTILINE)
    return re.sub(r'(?m)^ *#.*\n?', '', code)

## convert ipython to python

In [35]:
from nbformat import reads, NO_CONVERT
from nbconvert import PythonExporter
def code_extractor(jpt):
    nb = reads(jpt, NO_CONVERT)
    exporter = PythonExporter()
    source, meta = exporter.from_notebook_node(nb)
    return stripComments(source)

## calls Matchmaker and matches functions

In [36]:
import imp
Matchmaker = imp.load_source('name', '../matchmaker.py')
import ast
def function_matching(nb):
    tree = ast.parse(nb)
    mm = Matchmaker.Matchmaker()
    mm.visit(tree)
    matchs = mm.matching()
    return matchs

## for a file in a commit convert ipython to python, match the functions and write to disk

In [37]:
import difflib
import pandas as pd
def file_matcher(c,f,newfile):
    if f.new_path in paths[c.hash]:
        try:
            before = code_extractor(f.source_code_before)
        except TypeError:
            before = ""
        lines_b = before.strip().splitlines()
        new = code_extractor(f.source_code)
        lines_n = new.strip().splitlines()
        
        matches = function_matching(new)
        
        columns = ['repo','path','author','commit','line','code']
        temp_path = "../data/temp.csv"
        with open(temp_path, "w", encoding="utf8") as temp:
            writer = csv.DictWriter(temp, fieldnames=columns)
            writer.writeheader()
            if before != "":
                diff = difflib.unified_diff(lines_b, lines_n, fromfile='before', tofile='new', lineterm='', n=0)
                lines = list(diff)[2:]
                lineno = 0
                for line in lines:
                    prefix = '@@'
                    if line.startswith(prefix):
                        s = line[line.find("+"):]
                        try:
                            lineno = int(s[1:s.find(",")])
                        except:
                            lineno = int(s[1:s.find(" ")])
                    else:
                        if line.startswith("+"):
                            writer.writerow({"repo": c.project_name, 
                                             "path": f.new_path,
                                             "author": c.author.email,
                                             "commit": c.hash,
                                             "line": lineno,
                                             "code": line[1:]})
                            lineno += 1
            else:
                for idx,line in enumerate(lines_n):
                    writer.writerow({"repo": c.project_name, 
                                     "path": f.new_path,
                                     "author": c.author.email,
                                     "commit": c.hash,
                                     "line": idx+1,
                                     "code": line})
      
        changes = pd.read_csv(temp_path)
        changes['line'] = changes['line'].astype(int)
        matches['line'] = matches['line'].astype(int)
        result = pd.merge(matches, changes, how = 'inner', on= "line")
        result.to_csv(newfile, mode='a', header=False, index= False)

## load the repository locally stored and iterate through commits

In [38]:
def repo_matcher(repo, commits, paths, newfile):
    final_path = folder+repo
    repository = RepositoryMining(final_path, only_no_merge=True, only_commits=commits[r])
    for c in repository.traverse_commits():
        for f in c.modifications:
            try:
                file_matcher(c,f,newfile)
            except KeyboardInterrupt:
                break
            except Exception as e:
                continue

## load commit data 

In [29]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import os.path
from os import path

df = pd.read_csv('../data/commits_bigquery.csv')
df = df.drop(df.columns[0], axis = 1)

repos = df_all.repo.unique()
folder = "/home/ubuntu/repos/"
repos = repos.tolist()
for i in repos:
    if not path.exists(folder+i):
        repos.remove(i)


commits = df_all[['repo','commit']].groupby('repo')['commit'].apply(list)
paths = df_all[['commit','path']].groupby('commit')['path'].apply(list)

## iterate through commit data and call the functions above

In [39]:
from pydriller import RepositoryMining
import difflib
import pandas as pd
import csv
#inspect all repos and commits
fields = ['line','function','package','class','repo','path', 'author', 'commit','code']
to_disk = "../data/repo_function_matched.csv"
with open(to_disk, "w", encoding="utf8") as newfile:
    writer = csv.DictWriter(newfile, fieldnames=fields)
    writer.writeheader()
    
for r in repos:
    try:
        repo_matcher(r, commits, paths, to_disk)
    except KeyboardInterrupt:
        break
    except Exception as e:
        continue